<a href="https://colab.research.google.com/github/agam844/Active-Learning/blob/main/Active_Learning_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
mnist = fetch_openml('mnist_784')

# Split the data into features and labels
x = mnist.data
y = mnist.target.astype(int)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Training on the entire Dataset

In [ ]:
# Scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# Initialize the logistic regression model
model = LogisticRegression(max_iter=100)

# Train the model
model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.915


Active Learning on 10% of the dataset

In [ ]:
num_entries = min(6000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

In [ ]:
# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

In [ ]:
remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [ ]:
#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  for i in range(len(remaining_x_train)):
    # print("pred: ", y_pred[i])
    # print("original: ", remaining_y_train[i])
    # print("prob: ",  np.max(y_pred_proba, axis=1)[i] )
    if y_pred[i] != remaining_y_train[i]:
      np.max(y_pred_proba, axis=1)[i] = 0

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 1000
  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  additional_indices = np.argsort(np.max(y_pred_proba, axis=1))[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 1
Number of labeled samples: 7000
Number of remaining samples: 49000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 2
Number of labeled samples: 8000
Number of remaining samples: 48000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Iteration: 8
Number of labeled samples: 14000
Number of remaining samples: 42000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 9
Number of labeled samples: 15000
Number of remaining samples: 41000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 10
Number of labeled samples: 16000
Number of remaining samples: 40000


In [ ]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.970375


5K initial and adding 2K

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
#Initialising the enteries
num_entries = min(5000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)
# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]
#remaining Enteries
remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [ ]:
#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  for i in range(len(remaining_x_train)):
    # print("pred: ", y_pred[i])
    # print("original: ", remaining_y_train[i])
    # print("prob: ",  np.max(y_pred_proba, axis=1)[i] )
    if y_pred[i] != remaining_y_train[i]:
      np.max(y_pred_proba, axis=1)[i] = 0

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 2000
  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  additional_indices = np.argsort(np.max(y_pred_proba, axis=1))[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 1
Number of labeled samples: 7000
Number of remaining samples: 49000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 2
Number of labeled samples: 9000
Number of remaining samples: 47000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Iteration: 8
Number of labeled samples: 21000
Number of remaining samples: 35000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 9
Number of labeled samples: 23000
Number of remaining samples: 33000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 10
Number of labeled samples: 25000
Number of remaining samples: 31000


In [ ]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9859677419354839


10k initial adding 500

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
#Initialising the enteries
num_entries = min(10000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)
# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]
#remaining Enteries
remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [ ]:
#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  for i in range(len(remaining_x_train)):
    # print("pred: ", y_pred[i])
    # print("original: ", remaining_y_train[i])
    # print("prob: ",  np.max(y_pred_proba, axis=1)[i] )
    if y_pred[i] != remaining_y_train[i]:
      np.max(y_pred_proba, axis=1)[i] = 0

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 500
  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  additional_indices = np.argsort(np.max(y_pred_proba, axis=1))[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 1
Number of labeled samples: 10500
Number of remaining samples: 45500


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 2
Number of labeled samples: 11000
Number of remaining samples: 45000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Iteration: 8
Number of labeled samples: 14000
Number of remaining samples: 42000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 9
Number of labeled samples: 14500
Number of remaining samples: 41500


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 10
Number of labeled samples: 15000
Number of remaining samples: 41000


In [ ]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9442926829268292


Using uncertainity but not correctness

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
#Initialising the enteries
num_entries = min(6000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)
# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]
#remaining Enteries
remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [ ]:
#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 1000
  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  additional_indices = np.argsort(np.max(y_pred_proba, axis=1))[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 1
Number of labeled samples: 7000
Number of remaining samples: 49000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 2
Number of labeled samples: 8000
Number of remaining samples: 48000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Iteration: 8
Number of labeled samples: 14000
Number of remaining samples: 42000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 9
Number of labeled samples: 15000
Number of remaining samples: 41000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Iteration: 10
Number of labeled samples: 16000
Number of remaining samples: 40000


In [ ]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.970475
